# Import Libraries

In [5]:
import requests
from bs4 import BeautifulSoup
import csv
import os

# Scraping Function

#### I've supposed that the category is Audio & Hifi in Electronics, is page: https://www.bol.com/nl/nl/sf/electronics/ there are many categories (Audio & Hifi) is one of them

In [7]:
def main(page):
    src = page.content
    soup = BeautifulSoup(src, "lxml")
    
    #list to store dictionaries containing the product information
    prod_list = []
    
    #selects a list of products on the web page
    lists = soup.find("ul", {"class" : "list-view"})
    products = lists.find_all("li", {"class" : "product-item--row"})
    print("Number Of Products = ", len(products))
    
    #extracts information from the product
    def get_info(products):
        pid = products.get('data-id')
        #print(pid)
        
        info = products.find("div", {"class" : "product-item__info"})
        
        title = info.find("div", {"class" : "product-title--inline"}).text.strip()
        #print(title)
        
        brand = info.find("ul", {"class" : "product-creator"}).text.strip()
        #print(brand)
        
        #to handle errors
        try:
            rate = info.find("div", {"class" : "u-mb--xs"}).get('aria-label').split()[0]
            #print(rate)
        except:
            rate = ""
        
        n_reviews = info.find("div", {"class" : "star-rating"}).get('data-count')
        #print(n_reviews)
        
        
        url = "https://www.bol.com" + info.find("div", {"class" : "product-title--inline"}).find("a", {"class" : "product-title"}).get('href')
        #print(url)
        
        #to handle errors
        try:
            price_elem = products.find("span", {"class" : "promo-price"}).text.strip()
            price_fraction_elem = products.find("span", {"class" : "promo-price__fraction"}).text.strip()
            
            if price_fraction_elem == "-":
                price_fraction_elem = "00"
            
            #Concatenate the price and price fraction
            price = price_elem + ',' + price_fraction_elem if price_fraction_elem else price_elem
            #print(price)
        except:
            price=""
        
        category = soup.find("h1", {"class" : "bol_header"}).text.strip()
        #print(category)
        
        prod_list.append({"pid": pid, "title": title, "brand": brand, "rating": rate, "n_reviews":n_reviews, "url": url, "price": price, "category_name": category})
       
    #iterates over each product in the products list
    for product in products:
        get_info(product)
        
    #the path the file will be saved in
    path = "Products_Data.csv"
    
    #to check if the file is exists or not
    file_exists = os.path.isfile(path)
    
    #to extract the header
    fieldnames = prod_list[0].keys()

    with open(path, "a", errors='ignore', newline='') as file: #"a" for append
        writer = csv.DictWriter(file, fieldnames)

        #to write the header one time
        if not file_exists:
            writer.writeheader()

        writer.writerows(prod_list)    

# Function To Handle which page to scrape

In [3]:
def scrape(url, num):
    if (num == 1):
        page = requests.get(url)
    else:
        page = requests.get(url + "?page=" + str(num))
    main(page)
    print("Done")

In [8]:
page_num = input("Enter The Number of Page you want to scrape data from: ")
url = "https://www.bol.com/nl/nl/l/audio-hifi/10714/"

scrape(url, page_num)

Enter The Number of Page you want to scrape data from: 6
Number Of Products =  24
Done
